### Python-SQL Connection

In [1]:

import psycopg2

# "dbname=test user=postgres"
# dbname – the database name (database is a deprecated alias)
# user – user name used to authenticate
# password – password used to authenticate
# host – database host address (defaults to UNIX socket if not provided)
# port – connection port number (defaults to 5432 if not provided)
conn = psycopg2.connect(dbname = 'afyaproduction', 
                        user = '[enter your own user here]', 
                        password = '[enter your own password here]',
                        host = '34.72.27.131')

cursor = conn.cursor()


In [4]:
# 
# @dev I used global variable cursor
# 
def sql_to_df(name_of_table):
    import pandas as pd
    cursor.execute(f'SELECT * FROM public.{name_of_table};')
    d = cursor.fetchall()
    cursor.execute(f'SELECT * FROM public.{name_of_table} LIMIT 0;')
    c = [desc[0] for desc in cursor.description]
    _df = pd.DataFrame(data=d, columns=c)
    return _df

In [5]:
chat_sessions = sql_to_df('chat_sessions')

In [6]:
users = sql_to_df('users')

In [12]:
# Test on last session
test_chat_session = chat_sessions.loc[173]
# Pull information on last sessions

In [43]:
# Make a Patient Object
test_patient = make_patient('325bad90-b7fd-11eb-b94b-bdccd56f49bf')

In [47]:
test_patient.dob # 0-Female, 1-Male

'12-12-1994'

In [51]:
test_chat_session.index

Index(['id', 'status', 'close_codes', 'speciality', 'permissions', 'createdAt',
       'updatedAt', 'members', 'chief_complaint', 'owner_id_1', 'owner_id_2',
       'last_sender_name', 'last_sent_message', 'last_sent_type', 'read_by',
       'agora_ids', 'last_sender_id', 'updatedTime', 'patient_id',
       'patient_agora_id', 'org_id', 'ICD', 'CPT', 'is_archive',
       'close_code_date', 'language'],
      dtype='object')

In [41]:
#
# @dev 
# 
# input: Pandas Series for that 'row'
# output: Transaction Object
#
# need to define count, iday, itime, patient_obj, consult_id, responded_date,
#                 icd10_code, sr_last, sr_first, sr_npi, CCI_BLANK_or_MED
# 
# 
def make_transaction(chat_session):
    from datetime import date
    from datetime import datetime

    today = date.today()
    now = datetime.now()
    iday = today.strftime("%y%m%d")
    itime = now.strftime("%H%M")
    
    the_patient = make_patient(chat_session['patient_id'])
#
#
#
#
#

In [42]:
#
# @dev I used global variable (pandas dataframe) *users*
#
# input: patient_id from SQL database
# output: Patient Object defined
#                 
# Object Variables: self.last, self.first, self.id, self.address, self.city, self.state, self.zip, self.dob, self.gender
# 
# 
def make_patient(patient_id):
    ids = users['user_id']
    i = ids.tolist().index(patient_id)
    if(i != -1):
        u = users.loc[i]
    else:
        print('Could not find patient in user list')
    return Patient(u['last_name'], u['first_name'], None, u['address'], u['city'], u['state'], u['zipcode'], u['dob'], u['gender'])
#
# Very IMPORTANT
# That HPSJ_ID that is None right no gets defined later
#

In [8]:
cursor.close()
conn.close()

### 837P Formatting

In [20]:
import pandas as pd

df = pd.read_excel('Updated_HPSJ837PFieldDescriptionsMeltonEd.xlsx', engine='openpyxl')

In [21]:
df

,Segments,Description,Standard / Default Value,HPSJ's Value,Afya's Database Value,Example,Notes
0,ISA,Interchange Control Header,NaN,NaN,NaN,NaN,NaN
1,ISA00,Segment Name,ISA,NaN,Hard-coded,ISA,NaN
2,ISA01,Authorization Information Qualifer,"00, 03",00 (No Auth Info Present),Hard-coded,00,NaN
3,ISA02,Authorization Information,Required,{10 blank spaces},CLM_TRAD_PARTNER.ISA02,{10 blank spaces},NaN
4,ISA03,Security Information Qualifer,"00, 01",00 (No Security Info Present),Hard-coded,00,NaN
...,...,...,...,...,...,...,...
248,IEA,NaN,NaN,NaN,NaN,NaN,NaN
249,IEA00,Segment Name,IEA,NaN,Hard-coded,IEA,NaN
250,IEA01,Number of Included Functional Groups,Required,Count of # Functional Groups,COUNT (PROGRAMMATICALLY GENERATED),1,NaN
251,IEA02,Interchange Control Number,Required,ICN,CLM_TRAD_PARTNER.ICN,{interchange_control_number },NaN


In [22]:
def isnan(x):
    try:
        y = float(x)
        return math.isnan(y)
    except:
        return False

In [23]:
import math

test = []
arr = []
for e in df['Example']:
    if(isnan(e)):
        test.append(arr)
        arr = []
    else:
        if(e == '{BLANK}'):
            arr.append('')
        else:
            arr.append(e)


In [24]:
everything = test[1:]

In [25]:
envelope_start = everything[:2]
transaction_arr = everything[2:-2]
envelope_end = everything[-2:]

In [26]:
len(transaction_arr)

32

In [27]:
transaction = ''
newline = '\n'
for t in transaction_arr:
    for i,_t in enumerate(t):
        if i != (len(t)):
            transaction += str(_t)
            transaction += '*'
    transaction += f'~{newline}'

In [28]:
transaction

'ST*837*{count}*005010X222A1*~\nBHT*0019*00*0123*{iday}*{itime}*RP*~\nNM1*41*2*AFYA GLOBAL INC*****46*1982269031*~\nPER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~\nNM1*40*2*HPSJ*****46*00303*~\nHL*1**20*1*~\nPRV*BI*PXC*261QM1300X*~\nNM1*85*2*HUBMD*****XX*1982269031*~\nN3*104 E Olive Ave*Suite 100*~\nN4*Redlands*CA*92373-5255*~\nREF*EI*83-2774928*~\nPER*IC*Stanley Frencher*TE*9094559473*~\nHL*2*1*22*0*~\nSBR*S*18*******MC*~\nNM1*IL*1*{patient_last}*{patient_first}****MI*{hpsj_id}*~\nN3*{patient_address}**~\nN4*{patient_city}*{patient_state}*{patient_zip}*~\nDMG*D8*{patient_dob}*{patient_gender}*~\nNM1*PR*2*HPSJ*****PI*00303*~\nN3*7751 S Manthey Rd**~\nN4*French Camp*CA*95231*~\nCLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~\nDTP*304*D8*{responded_date}*~\nCN1*05*0*~\nHI*ABK:{icd10_code}*~\nNM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~\nSBR*P*18**{CCI_BLANK_or_MED}*****MC*~\nAMT*A8*0*~\nLX*1*~\nSV1*99451:GQ*34.50*UN*1*02***~\nDTP*472*D8*20200226*~\nSE*{num_of_lines}*{count}*~\n'

In [29]:
print(transaction)

ST*837*{count}*005010X222A1*~
BHT*0019*00*0123*{iday}*{itime}*RP*~
NM1*41*2*AFYA GLOBAL INC*****46*1982269031*~
PER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~
NM1*40*2*HPSJ*****46*00303*~
HL*1**20*1*~
PRV*BI*PXC*261QM1300X*~
NM1*85*2*HUBMD*****XX*1982269031*~
N3*104 E Olive Ave*Suite 100*~
N4*Redlands*CA*92373-5255*~
REF*EI*83-2774928*~
PER*IC*Stanley Frencher*TE*9094559473*~
HL*2*1*22*0*~
SBR*S*18*******MC*~
NM1*IL*1*{patient_last}*{patient_first}****MI*{hpsj_id}*~
N3*{patient_address}**~
N4*{patient_city}*{patient_state}*{patient_zip}*~
DMG*D8*{patient_dob}*{patient_gender}*~
NM1*PR*2*HPSJ*****PI*00303*~
N3*7751 S Manthey Rd**~
N4*French Camp*CA*95231*~
CLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~
DTP*304*D8*{responded_date}*~
CN1*05*0*~
HI*ABK:{icd10_code}*~
NM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~
SBR*P*18**{CCI_BLANK_or_MED}*****MC*~
AMT*A8*0*~
LX*1*~
SV1*99451:GQ*34.50*UN*1*02***~
DTP*472*D8*20200226*~
SE*{num_of_lines}*{count}*~



In [30]:
class Transaction:
    
    # *** num of lines is currently 32 ***
    
    num_of_lines = 32
    
    def __init__(self, count, iday, itime, patient_obj, consult_id, responded_date,
                 icd10_code, sr_last, sr_first, sr_npi, CCI_BLANK_or_MED):
        self.data = f'ST*837*{count}*005010X222A1*~\nBHT*0019*00*0123*{iday}*{itime}*RP*~\nNM1*41*2*AFYA GLOBAL INC*****46*1982269031*~\nPER*IC*AFYA SUPPORT*TE*9493995380*EM*admin@afyachat.com*~\nNM1*40*2*HPSJ*****46*00303*~\nHL*1**20*1*~\nPRV*BI*PXC*261QM1300X*~\nNM1*85*2*HUBMD*****XX*1982269031*~\nN3*104 E Olive Ave*Suite 100*~\nN4*Redlands*CA*92373-5255*~\nREF*EI*83-2774928*~\nPER*IC*Stanley Frencher*TE*9094559473*~\nHL*2*1*22*0*~\nSBR*S*18*******MC*~\nNM1*IL*1*{patient_obj.last}*{patient_obj.first}****MI*{patient_obj.id}*~\nN3*{patient_obj.address}**~\nN4*{patient_obj.city}*{patient_obj.state}*{patient_obj.zip}*~\nDMG*D8*{patient_obj.dob}*{patient_obj.gender}*~\nNM1*PR*2*HPSJ*****PI*00303*~\nN3*7751 S Manthey Rd**~\nN4*French Camp*CA*95231*~\nCLM*{consult_id}*34.50***02:B:01*Y*A*Y*Y*~\nDTP*304*D8*{responded_date}*~\nCN1*05*0*~\nHI*ABK:{icd10_code}*~\nNM1*82*1*{sr_last}*{sr_first}****XX*{sr_npi}*~\nSBR*P*18**{CCI_BLANK_or_MED}*****MC*~\nAMT*A8*0*~\nLX*1*~\nSV1*99451:GQ*34.50*UN*1*02***~\nDTP*472*D8*20200226*~\nSE*{num_of_lines}*{count}*~\n'

In [31]:
class Patient:
    def __init__(self, patient_last, patient_first, hpsj_id, 
                 patient_address, patient_city, patient_state, patient_zip, 
                 patient_dob, patient_gender):
        self.last = patient_last
        self.first = patient_first
        self.id = hpsj_id
        self.address = patient_address
        self.city = patient_city
        self.state = patient_state
        self.zip = patient_zip
        self.dob = patient_dob
        self.gender = patient_gender